# Problem 3

Use this notebook to write your code for problem 3.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3D - Convolutional network

As in problem 2, we have conveniently provided for your use code that loads and preprocesses the MNIST data.

In [2]:
# load MNIST data into PyTorch format
import torch
import torchvision
import torchvision.transforms as transforms

# set batch size
batch_size = 32

# load training data downloaded into data/ folder
mnist_training_data = torchvision.datasets.MNIST('data/', train=True, download=True,
                                                transform=transforms.ToTensor())
# transforms.ToTensor() converts batch of images to 4-D tensor and normalizes 0-255 to 0-1.0
training_data_loader = torch.utils.data.DataLoader(mnist_training_data,
                                                  batch_size=batch_size,
                                                  shuffle=True)

# load test data
mnist_test_data = torchvision.datasets.MNIST('data/', train=False, download=True,
                                                transform=transforms.ToTensor())
test_data_loader = torch.utils.data.DataLoader(mnist_test_data,
                                                  batch_size=batch_size,
                                                  shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [3]:
# look at the number of batches per epoch for training and validation
print(f'{len(training_data_loader)} training batches')
print(f'{len(training_data_loader) * batch_size} training samples')
print(f'{len(test_data_loader)} validation batches')

1875 training batches
60000 training samples
313 validation batches


In [4]:
# sample model
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.5),
    
    nn.Conv2d(8, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.5),
    
    nn.Flatten(),
    nn.Linear(25*8, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
)

In [5]:
# why don't we take a look at the shape of the weights for each layer
for p in model.parameters():
    print(p.data.shape)

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8, 8, 3, 3])
torch.Size([8])
torch.Size([64, 200])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [6]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 14178


In [7]:
# For a multi-class classification problem
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

In [16]:
for i, data in enumerate(training_data_loader):
    images, labels = data
    print(images.shape)
    print(labels.shape)
    break;

torch.FloatTensor
torch.LongTensor


In [ ]:
# Train the model for 10 epochs, iterating on the data in batches
n_epochs = 10

# store metrics
training_accuracy_history = np.zeros([n_epochs, 1])
training_loss_history = np.zeros([n_epochs, 1])
validation_accuracy_history = np.zeros([n_epochs, 1])
validation_loss_history = np.zeros([n_epochs, 1])

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/10:', end='')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[epoch] /= len(training_data_loader)
    training_accuracy_history[epoch] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(test_data_loader)
        validation_accuracy_history[epoch] = test_correct / test_total
    print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

Epoch 1/10:...........
	loss: 0.6070, acc: 0.8065, val loss: 0.2247, val acc: 0.9403
Epoch 2/10:...........
	loss: 0.4049, acc: 0.8734, val loss: 0.1784, val acc: 0.9541
Epoch 3/10:...........
	loss: 0.3736, acc: 0.8844, val loss: 0.1675, val acc: 0.9497
Epoch 4/10:...........
	loss: 0.3626, acc: 0.8884, val loss: 0.1427, val acc: 0.9552
Epoch 5/10:...........
	loss: 0.3522, acc: 0.8919, val loss: 0.1531, val acc: 0.9598
Epoch 6/10:...........
	loss: 0.3604, acc: 0.8915, val loss: 0.2104, val acc: 0.9445
Epoch 7/10:...........
	loss: 0.3465, acc: 0.8964, val loss: 0.1373, val acc: 0.9609
Epoch 8/10:...........
	loss: 0.3503, acc: 0.8944, val loss: 0.2146, val acc: 0.9381
Epoch 9/10:...........
	loss: 0.3497, acc: 0.8949, val loss: 0.2138, val acc: 0.9454
Epoch 10/10:...........
	loss: 0.3524, acc: 0.8949, val loss: 0.1398, val acc: 0.9610


Above, we output the training loss/accuracy as well as the validation loss and accuracy. Not bad! Let's see if you can do better.

## 3G

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=(3,3), padding=(1, 1)),
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),
    
    nn.Conv2d(8, 8, kernel_size=(3,3), padding=(1, 1)),
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.1),
    
    nn.Flatten(),
    nn.Linear(392, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
)

In [ ]:
print(model)

Sequential(
  (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Dropout(p=0.1, inplace=False)
  (5): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Dropout(p=0.1, inplace=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=392, out_features=64, bias=True)
  (12): ReLU()
  (13): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 26498


In [ ]:
# For a multi-class classification problem
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

In [ ]:
# Train the model for 10 epochs, iterating on the data in batches
n_epochs = 10

# store metrics
training_accuracy_history = np.zeros([n_epochs, 1])
training_loss_history = np.zeros([n_epochs, 1])
validation_accuracy_history = np.zeros([n_epochs, 1])
validation_loss_history = np.zeros([n_epochs, 1])

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/10:', end='')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[epoch] /= len(training_data_loader)
    training_accuracy_history[epoch] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(test_data_loader)
        validation_accuracy_history[epoch] = test_correct / test_total
    print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

Epoch 1/10:...........
	loss: 0.2415, acc: 0.9319, val loss: 0.0654, val acc: 0.9799
Epoch 2/10:...........
	loss: 0.1021, acc: 0.9686, val loss: 0.0617, val acc: 0.9818
Epoch 3/10:...........
	loss: 0.0867, acc: 0.9732, val loss: 0.0673, val acc: 0.9792
Epoch 4/10:...........
	loss: 0.0769, acc: 0.9762, val loss: 0.0502, val acc: 0.9857
Epoch 5/10:...........
	loss: 0.0727, acc: 0.9782, val loss: 0.0578, val acc: 0.9835
Epoch 6/10:...........
	loss: 0.0700, acc: 0.9790, val loss: 0.0508, val acc: 0.9865
Epoch 7/10:...........
	loss: 0.0675, acc: 0.9795, val loss: 0.0481, val acc: 0.9838
Epoch 8/10:...........
	loss: 0.0657, acc: 0.9808, val loss: 0.0494, val acc: 0.9850
Epoch 9/10:...........
	loss: 0.0666, acc: 0.9797, val loss: 0.0452, val acc: 0.9857
Epoch 10/10:...........
	loss: 0.0625, acc: 0.9812, val loss: 0.0438, val acc: 0.9863


In [ ]:
for ps in np.linspace(0,1,11):
  model = nn.Sequential(
      nn.Conv2d(1, 8, kernel_size=(3,3), padding=(1, 1)),
      nn.BatchNorm2d(8),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Dropout(p=ps),
      
      nn.Conv2d(8, 8, kernel_size=(3,3), padding=(1, 1)),
      nn.BatchNorm2d(8),
      nn.ReLU(),
      nn.MaxPool2d(2),
      nn.Dropout(p=ps),
      
      nn.Flatten(),
      nn.Linear(392, 64),
      nn.ReLU(),
      nn.Linear(64, 10)
      # PyTorch implementation of cross-entropy loss includes softmax layer
  )
  count = 0
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.RMSprop(model.parameters())
  n_epochs = 1
  # store metrics
  training_accuracy_history = np.zeros([n_epochs, 1])
  training_loss_history = np.zeros([n_epochs, 1])
  validation_accuracy_history = np.zeros([n_epochs, 1])
  validation_loss_history = np.zeros([n_epochs, 1])

  for epoch in range(n_epochs):
      print(f'P = {ps}:', end='\n')
      print(f'Epoch {epoch+1}/1:', end='')
      train_total = 0
      train_correct = 0
      # train
      model.train()
      for i, data in enumerate(training_data_loader):
          images, labels = data
          optimizer.zero_grad()
          # forward pass
          output = model(images)
          # calculate categorical cross entropy loss
          loss = criterion(output, labels)
          # backward pass
          loss.backward()
          optimizer.step()
          
          # track training accuracy
          _, predicted = torch.max(output.data, 1)
          train_total += labels.size(0)
          train_correct += (predicted == labels).sum().item()
          # track training loss
          training_loss_history[epoch] += loss.item()
          # progress update after 180 batches (~1/10 epoch for batch size 32)
          if i % 180 == 0: print('.',end='')
      training_loss_history[epoch] /= len(training_data_loader)
      training_accuracy_history[epoch] = train_correct / train_total
      print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
          
      # validate
      test_total = 0
      test_correct = 0
      with torch.no_grad():
          model.eval()
          for i, data in enumerate(test_data_loader):
              images, labels = data
              # forward pass
              output = model(images)
              # find accuracy
              _, predicted = torch.max(output.data, 1)
              test_total += labels.size(0)
              test_correct += (predicted == labels).sum().item()
              # find loss
              loss = criterion(output, labels)
              validation_loss_history[epoch] += loss.item()
          validation_loss_history[epoch] /= len(test_data_loader)
          validation_accuracy_history[epoch] = test_correct / test_total
      print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

P = 0.0:
Epoch 1/1:...........
	loss: 0.2236, acc: 0.9425, val loss: 0.0869, val acc: 0.9724
P = 0.1:
Epoch 1/1:...........
	loss: 0.2593, acc: 0.9310, val loss: 0.0690, val acc: 0.9793
P = 0.2:
Epoch 1/1:...........
	loss: 0.2853, acc: 0.9184, val loss: 0.0985, val acc: 0.9693
P = 0.30000000000000004:
Epoch 1/1:...........
	loss: 0.3389, acc: 0.9054, val loss: 0.0863, val acc: 0.9745
P = 0.4:
Epoch 1/1:...........
	loss: 0.4118, acc: 0.8715, val loss: 0.1294, val acc: 0.9581
P = 0.5:
Epoch 1/1:...........
	loss: 0.7135, acc: 0.7641, val loss: 0.1598, val acc: 0.9533
P = 0.6000000000000001:
Epoch 1/1:...........
	loss: 0.5981, acc: 0.8104, val loss: 0.1805, val acc: 0.9493
P = 0.7000000000000001:
Epoch 1/1:...........
	loss: 0.8608, acc: 0.7143, val loss: 0.2968, val acc: 0.9290
P = 0.8:
Epoch 1/1:...........
	loss: 1.2844, acc: 0.5617, val loss: 0.5077, val acc: 0.8759
P = 0.9:
Epoch 1/1:...........
	loss: 1.7395, acc: 0.3924, val loss: 1.2911, val acc: 0.6031
P = 1.0:
Epoch 1/1:.....